# Python для сбора данных

*Алла Тамбовцева, НИУ ВШЭ*

### Web-scraping

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача: выгрузить недавние новости в датафрейм pandas, чтобы потом сохранить все в файл Excel.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем ее:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`.

In [2]:
url = "https://nplus1.ru/"
page = requests.get(url)

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page

<Response [200]>

Объект `page` имеет тип `Response` и скрыт от наших глаз. Однако при его вызове мы видим число 200 – это код результата, который означает, что страница благополучно загружена.

У объекта типа `Response` есть атрибут `.text`, в котором хранится исходный код страницы, который мы можем посмотреть, нажав *Ctrl+U* в Chrome:

In [5]:
page.text

Результат выше – это обычная строка, тип *string*. Выполнять поиск по такой строке неудобно, поэтому загрузим из модуля `bs4` функцию `BeautifulSoup()`, которая позволит преобразовать эту строку в объект, который позволяет выполнять поиск по тегам.

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.text)

Если код выше выдает ошибку (зависит от версии `bs4`), можно указать парсер, который необходимо использовать, явно:

In [5]:
soup = BeautifulSoup(page.text, 'lxml')  #lxml

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [6]:
for link in soup.find_all('a'):
    print(link.get('href'))

#
/
#
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
/theme/explainatorium
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
/
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
#
/theme/explainatorium
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
https://nplus1.ru/blog/2020/04/17/how-music-works
https://nplus1.ru/blog/2020/04/17/how-music-works
https://nplus1.ru/blog/2020/04/17/nudity-censorship
https://nplus1.ru/blog/2020/04/16/stories-of-surgery-for-broken-hearts
https://nplus1.ru/material/2020/04/08/coronarumors
https://nplus1.ru/blog/2020/04/13/ancient-greece-from-prehistoric-to-hellenistic-tim
https://nplus1.ru/blog/2020/04/10/guestmixIvanZoloto
https://nplus1.ru/blog/2020/04/10/the-nature-and-necessity-of-bees
https://nplus1.ru/blog/2020/04/07/troubled-oculudentavis
https://nplus1.ru/blog/2020/04/06/maps-of-meaning
https://nplus1.ru/material/2020/04/08/coronarumors
/news/2020/04/27/venus-atmosphere-tidal-wav

В коде выше мы использовали метод `find_all()`, который выполняет поиск по заданному тэгу и возвращает список частей кода HTML с выбранным тэгом. Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a href="/rubric/robots-drones" class="">` изнутри выглядит как словарь следующего вида:

        {'href' : '/rubric/robots-drones', 
         'class' : ''}.
         
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Именно его мы и использовали в коде выше, чтобы извлечь содержимое `href`. 

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [7]:
urls = []
for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        urls.append(link.get('href'))
urls

['/news/2020/04/27/venus-atmosphere-tidal-waves',
 '/news/2020/04/27/zumwalt',
 '/news/2020/04/27/starship-pressure-passed',
 '/news/2020/04/27/e-fan-x',
 '/news/2020/04/25/hyperphagia-neurons',
 '/news/2020/04/25/dna-pocket',
 '/news/2020/04/25/moon-landing-spray',
 '/news/2020/04/25/thermogalvanic-hydrogel-cooling',
 '/news/2020/04/24/30-years-hubble',
 '/news/2020/04/24/feline-grimace-scale',
 '/news/2020/04/24/supermassive-black-hole-escape-star',
 '/news/2020/04/24/adobe',
 '/news/2020/04/24/broken-hearts',
 '/news/2020/04/24/00s-taxi',
 '/news/2020/04/24/cold-sweet',
 '/news/2020/04/24/avian',
 '/news/2020/04/24/kentaurs-interstellar',
 '/news/2020/04/24/ocean-macroplastic-finding',
 '/news/2020/04/24/smartwatch',
 '/news/2020/04/24/antarctic-calyptocephalella',
 '/news/2020/04/23/underwater-quantum-communication',
 '/news/2020/04/21/self-adapt-PVDF',
 '/news/2020/04/27/venus-atmosphere-tidal-waves',
 '/news/2020/04/25/thermogalvanic-hydrogel-cooling',
 '/news/2020/04/25/hyperpha

Ссылки, которые у нас есть в списке `urls`, относительные: они неполные, начало ссылки, в данном случае название сайта, отсутствует. Давайте превратим их в абсолютные ‒ склеим с ссылкой https://nplus1.ru.

In [8]:
full_urls = ['https://nplus1.ru' + u for u in urls]
full_urls

['https://nplus1.ru/news/2020/04/27/venus-atmosphere-tidal-waves',
 'https://nplus1.ru/news/2020/04/27/zumwalt',
 'https://nplus1.ru/news/2020/04/27/starship-pressure-passed',
 'https://nplus1.ru/news/2020/04/27/e-fan-x',
 'https://nplus1.ru/news/2020/04/25/hyperphagia-neurons',
 'https://nplus1.ru/news/2020/04/25/dna-pocket',
 'https://nplus1.ru/news/2020/04/25/moon-landing-spray',
 'https://nplus1.ru/news/2020/04/25/thermogalvanic-hydrogel-cooling',
 'https://nplus1.ru/news/2020/04/24/30-years-hubble',
 'https://nplus1.ru/news/2020/04/24/feline-grimace-scale',
 'https://nplus1.ru/news/2020/04/24/supermassive-black-hole-escape-star',
 'https://nplus1.ru/news/2020/04/24/adobe',
 'https://nplus1.ru/news/2020/04/24/broken-hearts',
 'https://nplus1.ru/news/2020/04/24/00s-taxi',
 'https://nplus1.ru/news/2020/04/24/cold-sweet',
 'https://nplus1.ru/news/2020/04/24/avian',
 'https://nplus1.ru/news/2020/04/24/kentaurs-interstellar',
 'https://nplus1.ru/news/2020/04/24/ocean-macroplastic-findin

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [9]:
url0 = full_urls[0]
print(url0)

https://nplus1.ru/news/2020/04/27/venus-atmosphere-tidal-waves


In [10]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [11]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="black" name="apple-mobile-web-app-status-bar-style"/>,
 <meta content="2020-04-27" itemprop="datePublished"/>,
 <meta content="Кристина Уласович" name="mediator_author"/>,
 <meta content="Японские планетологи выяснили, что необычно быстрое вращение атмосферы Венеры поддерживается благодаря тепловым приливам, волнам Россби и турбулентности" name="description"/>,
 <meta content="Кристина Уласович" name="author"/>,
 <meta content="" name="copyright"/>,
 <meta content="Суперротацию атмосферы Венеры объяснили тепловыми приливами" property="og:title"/>,
 <meta content="https://nplus1.ru/images/2020/04/27/d7e9e0accb014ce8a17f20391ee1e50a.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2020/04/27/venus-atmosphere-tidal-waves" property="og:url"/>,
 

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута:

In [12]:
soup0.find_all('meta', {'name' : 'author'})

[<meta content="Кристина Уласович" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [13]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Кристина Уласович" name="author"/>

Объект выше имеет структуру как у словаря, поэтому мы можем вызвать значение `content` через метод `.get()`:

In [14]:
soup0.find_all('meta', {'name' : 'author'})[0].get('content')  

'Кристина Уласович'

Или вовсе без него, указав название ключа в квадратных скобках:

In [15]:
author = soup0.find_all('meta', {'name' : 'author'})[0]['content']

Изучим внимательно исходный код страницы новости и аналогичным образом найдем заголовок новости, описание и дату публикации:

In [16]:
title = soup0.find_all('meta', 
                       {'property' : 'og:title'})[0]['content']
description = soup0.find_all('meta', 
                             {'property' : 'og:description'})[0]['content']
date = soup0.find_all('meta', 
                      {'itemprop' : 'datePublished'})[0]['content']

Осталось вытащить сложность текста и рубрики. Сложность находится в тэге `span` с классом `difficult-value`:

In [17]:
soup0.find_all('span', {'class' : 'difficult-value'})[0]

<span class="difficult-value">5.7</span>

Извлечем текст, который находится внутри тэгов, с помощью атрибута `.text`:

In [18]:
difficult = soup0.find_all('span', 
                           {'class' : 'difficult-value'})[0].text

С рубриками интереснее. Рубрики находятся в тэгах `p` с классом `table`:

In [19]:
soup0.find_all('p', {'class' : 'table'})[0]

<p class="table">
<a data-rubric="astronomy" href="/rubric/astronomy">Астрономия</a>
</p>

В этом отрывке кода есть ссылки на рубрики. «Выцепим» все ссылки по тэгу `a`:

In [20]:
raw_rubrics = soup0.find_all('p', 
                             {'class' : 'table'})[0].find_all('a')
raw_rubrics

[<a data-rubric="astronomy" href="/rubric/astronomy">Астрономия</a>]

А теперь извлечем из каждого кусочка кода для ссылки текст:

In [21]:
rubrics = []
for r in raw_rubrics:
    rubrics.append(r.text)
rubrics

['Астрономия']

Перейдем к самому главному – тексту новости. Как можно заметить, текст сохранен в абзацах `<p></p>`, причем безо всяких атрибутов. Сообщим Python, что нас интересуют куски с пустым атрибутом `class`:

In [22]:
soup0.find_all('p', {'class' : None})

[<p>Японские планетологи выяснили, что необычно быстрое вращение атмосферы Венеры поддерживается благодаря тепловым приливам, волнам Россби и турбулентности. К такому выводу они пришли на основе снимков облачного слоя, сделанных с помощью межпланетного зонда «Акацуки». Статья <a href="https://science.sciencemag.org/cgi/doi/10.1126/science.aaz4439" rel="nofollow" target="_blank">опубликована</a> в журнале <i>Science</i>.<br/></p>,
 <p>Еще в середине прошлого века астрономы заметили, что верхние слои плотного облачного покрова Венеры движутся намного быстрее ее поверхности. В то время как период вращения планеты составляет 243 земных дня, ее атмосфере на полный оборот требуется всего 92 часа — этот феномен назвали суперротацией. Для поддержания суперротации необходимо непрерывное перераспределение углового момента, которое позволило бы преодолеть трение с поверхностью планеты, однако механизмы, лежащие в основе этого процесса, до сих пор оставались неизвестны. </p>,
 <p>Такеши Хоринучи (

«Выцепим» все тексты (без тэгов) из полученного списка и склеим все элементы списка `text` через пробел:

In [23]:
pars = []
for p in soup0.find_all('p', {'class' : None}):
    pars.append(p.text)
text = " ".join(pars)

Избавимся от лишнего текста после фразы *Нашли опечатку?* и заменим лишние символы на обычные пробелы:

In [24]:
text_final = text.split("Нашли опечатку?")[0].replace('\xa0', 
                                                      ' ').replace('\n', ' ')

In [25]:
print(text_final)

Японские планетологи выяснили, что необычно быстрое вращение атмосферы Венеры поддерживается благодаря тепловым приливам, волнам Россби и турбулентности. К такому выводу они пришли на основе снимков облачного слоя, сделанных с помощью межпланетного зонда «Акацуки». Статья опубликована в журнале Science. Еще в середине прошлого века астрономы заметили, что верхние слои плотного облачного покрова Венеры движутся намного быстрее ее поверхности. В то время как период вращения планеты составляет 243 земных дня, ее атмосфере на полный оборот требуется всего 92 часа — этот феномен назвали суперротацией. Для поддержания суперротации необходимо непрерывное перераспределение углового момента, которое позволило бы преодолеть трение с поверхностью планеты, однако механизмы, лежащие в основе этого процесса, до сих пор оставались неизвестны.  Такеши Хоринучи (Takeshi Horinouchi) из Университета Хоккайдо вместе с коллегами изучили снимки, сделанные аппаратом «Акацуки» японского аэрокосмического агент

Теперь все красиво. Перейдем к семинару – напишем функцию для выгрузки информации по одной новости и применим ее к новостям на главной странице.

In [21]:
def get_info(url0):
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    author = soup0.find_all('meta', {'name' : 'author'})[0]['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0]['content']
    description = soup0.find_all('meta', 
                             {'property' : 'og:description'})[0]['content']
    date = soup0.find_all('meta', 
                      {'itemprop' : 'datePublished'})[0]['content']
    difficult = soup0.find_all('span', 
                           {'class' : 'difficult-value'})[0].text
    raw_rubrics = soup0.find_all('p', 
                             {'class' : 'table'})[0].find_all('a')
    rubrics = []
    for r in raw_rubrics:
        rubrics.append(r.text)
        
    pars = []
    for p in soup0.find_all('p', {'class' : None}):
        pars.append(p.text)
    text = " ".join(pars)
    
    text_final = text.split("Нашли опечатку?")[0].replace('\xa0', 
                                                      ' ').replace('\n', ' ')
    return [author, title, description, date, difficult, 
           rubrics, text_final] 